## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df=city_data_df.set_index("City_ID")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
City_ID,,,,,,,,,
0,Chokurdakh,RU,70.6333,147.9167,-16.91,86,95,3.65,overcast clouds
1,Lima,PE,-12.0432,-77.0282,68.90,76,0,1.01,clear sky
2,Ushuaia,AR,-54.8000,-68.3000,55.06,67,40,4.61,scattered clouds
3,Klaksvik,FO,62.2266,-6.5890,40.12,91,100,31.76,overcast clouds
4,Tasiilaq,GL,65.6145,-37.6368,12.36,70,77,4.65,broken clouds


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

What is the maximum temperature you would like for your trip? 80
What is the minimum temperature you would like for your trip? 70


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
City_ID,,,,,,,,,
12,Rikitea,PF,-23.1203,-134.9692,77.88,73,94,14.07,overcast clouds
17,Muscat,OM,23.6139,58.5922,73.42,49,0,4.12,clear sky
19,Hobart,AU,-42.8794,147.3294,72.27,70,75,1.01,broken clouds
22,Digha,IN,21.6833,87.5500,71.17,35,1,10.40,clear sky
24,Adrar,MR,20.5022,-10.0711,70.92,12,100,3.51,overcast clouds
27,Sao Joao Da Barra,BR,-21.6403,-41.0511,75.31,90,67,16.24,broken clouds
33,New Norfolk,AU,-42.7826,147.0587,71.91,51,92,3.00,overcast clouds
37,Atambua,ID,-9.1061,124.8925,78.87,80,100,7.83,overcast clouds
38,Atuona,PF,-9.8000,-139.0333,77.61,78,12,13.20,few clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   119
Country                118
Lat                    119
Lng                    119
Max Temp               119
Humidity               119
Cloudiness             119
Wind Speed             119
Weather Description    119
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
12,Rikitea,PF,77.88,overcast clouds,-23.1203,-134.9692,
17,Muscat,OM,73.42,clear sky,23.6139,58.5922,
19,Hobart,AU,72.27,broken clouds,-42.8794,147.3294,
22,Digha,IN,71.17,clear sky,21.6833,87.5500,
24,Adrar,MR,70.92,overcast clouds,20.5022,-10.0711,
27,Sao Joao Da Barra,BR,75.31,broken clouds,-21.6403,-41.0511,
33,New Norfolk,AU,71.91,overcast clouds,-42.7826,147.0587,
37,Atambua,ID,78.87,overcast clouds,-9.1061,124.8925,
38,Atuona,PF,77.61,few clouds,-9.8000,-139.0333,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("",np.nan, inplace=True)
clean_hotel_df=hotel_df.dropna()

In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))